# Derivativos de Ações e Commodities
##  Precificação com o Método de Diferenças Finitas
<sub>Uirá Caiado. 29 de Junho, 2016<sub>

#### Resumo

_Neste projeto vou implementar o método de diferenças finitas explícito para precificar diferentes derivativos. Como o preço de opções geralmente é descrito por equações diferenciais de difusão ou parabólicas, o método de diferenças finitas se mostra adequado, uma vez que é utilizado justamente para encontrar soluções numéricas para equações diferencias. Depois e implementar o modelo, vou compará-lo com os resultados obtidos pela solução analítica de cada instrumento._

## 1. Introdução

Nesta seção dou uma breve descrição sobre o método de diferenças finitas e declaro o problema que será abordado.

### 1.1. O Método Utilizado

Como colocado por Wilmott, encontrar soluções fechadas para precificação de muitos derivativos pode ser difícil, ou mesmo inviável. Como o preço destes instrumentos frequentemente são descritos por equações diferencias parciais, frequentemente utilizam-se métodos numéricos para encontrar uma solução, como [árvores binomiais](https://en.wikipedia.org/wiki/Binomial_options_pricing_model), simulação de [monte carlo](https://en.wikipedia.org/wiki/Monte_Carlo_method) e diferenças finitas.

O método de diferenças finitas encontra o valor de um derivativo calculando-o em todo domínio (factivel) de preços do instrumento base, incluíndo a passagem de tempo até seu vencimento.  Assim, são similares às árvores binomiais. Porém, ao invéz e discretizar os preços do ativo e a passagem do tempo em uma estrutura de árvore, discretiza em um grid.

Diferenças finitas também são muitos boas para lidar com problemas com poucas dimensões e equações diferenciais não lineares (o preço e tempo já são duas, e o Wilmott sugere até quatro). Para muitas dimensões, a implementação começa a ficar complicada e pouco eficiente, coisa que o Monte Carlo lida melhor. Porém, se há exercício antecipado e não linearidade, o método de diferenças finitas acaba se mostrando como solução mais viável.

### 1.2. O Problema

Considerando um ativo-objeto cuja dinâmica do preço satisfaz a seguinte EDE:

$$\frac{\mathrm{d} S_t}{S_t}=\mu\cdot \mathrm{d}t + \sigma\cdot \mathrm{d}W_{t}$$

Deseja-se calcular o preço justo de um derivativo um derivativo com característica europeia cujo payoff é descrito por uma função qualquer $V_T=V(T, S_T)$, onde $T$ é o vencimento do derivativo. $S_T$ é o preço do ativo-objeto em $T$ e é possível negociar qualquer quantidade dele em qualquer instante. Não há custo de transação (corretagem, emolumento, bid-ask spread, etc) e posições vendidas a descoberto no subjacente são permitidas, não havendo custos associados.

Pede-se que se implemente o algoritmo de diferenças finitas explícito e se calcule o (a) preço justo, (b) o Delta e (c) o Gamma de cada instrumento abaixo. Os valores encontrados devem ser comparados com o resultado de suas expressões analítica. A simulação deve ser feita para os payoffs abaixo. $K$ é o Strike da opção.
- $V(T, S_T)=ln(S_T)$
- $V(T, S_T)=(ln(S_T))^2$
- $V(T, S_T)=(S_T-K)^2$
- $V(T, S_T)=\mathbf{1}_{S_T > K}$
- $V(T, S_T)=max(S_T-K, 0)$

##  2. Diferenças Finitas

Abaixo, apresentarei os elementos necessários para implementar o método diferenças finitas, sendo eles: a [diferenciação](https://pt.wikipedia.org/wiki/Diferenciação_numérica) das derivadas parciais necessárias usando o Grid; a discretização da condição final para cada derivativo; e suas respectivas condições de contorno (nos limites do grid).

### 2.1. Diferenciação no Grid

De acordo com Wilmott, o grid utilizado pelo método de diferenças finitas tem passos de tempo e de preço (ou log do preço) geralmente homogênios. Porém, **não há** restrição para a forma do Grid, desde que os ajustes necessários sejam feitos. Como exposto por 

Considerando que podemos discretizar $t$ como $t=T-k\delta t$ e do $S$ como $S = i\delta S$, onde $i$ e $k$ são seus respectivos passos no grid, podemos escrever o valor da opção em cada ponto do grig como sendo:

$$V^{k}_{i} = ( i \delta S, T-k \delta t )$$

Seguindo notas de aula, aplicando Taylor ao preço $V(T, S_T)$ de um derivativo genérico, posso escrever a seguinte equação diferencial parabólica:

$$\frac{\partial V}{\partial t} + a(S, t) \cdot \frac{\partial V^2}{\partial S^2} + b(S, t) \cdot \frac{\partial V}{\partial S} + c(S, t)\cdot V = 0 $$

Sendo que $\frac{\partial V}{\partial t}$ também é chamado de theta ($\theta$), $\frac{\partial V}{\partial S}$ de delta ($\Delta$) e $\frac{\partial V^2}{\partial S^2}$ de gamma ($\Gamma$). Como demostrado por Wilmott, como a definição da primeira derivada $V$ em relação a $t$ é dado por

$$\frac{\partial V}{\partial t} = \underset{h \, \rightarrow \, 0}{\lim}\, \frac{V(S, t + h) - V(S, t)}{h}$$

Então podemos aproximar $\theta$ como sendo $\frac{\partial V}{\partial t} (S, t) \approx \frac{V_{i}^{k} - V_{i}^{k+1} }{\delta t}$. Note que $i$, que é relacionado ao passo do ativo, ficou fixo. *Também estou ignorando os erros de aproximação, como ignorarei em todas as outras diferenciações*.

O mesmo raciocínio pode ser utilizado para aproximar o $\Delta$. Porém, Wilmott ainda sugere que se utilize a [diferença centrada](http://math.stackexchange.com/questions/888259/can-someone-explain-in-general-what-a-central-difference-formula-is-and-what-it), que é dada por $\frac{\partial V}{\partial S} (S, t) \approx \frac{V_{i+1}^{k} - V_{i-1}^{k} }{2 \delta S}$. A discretização anterior também poderia ser utilizada, mas esta oferece um erro de aproximação menor. O único problema desta abordagem é que preciso saber os valores $S + \delta S$ e $S - \delta S$ e nas fronteiras do Grid não terei esta informação. Por tanto, nestes casos utilizarei a discretrização utilizando apenas um dos lados.

Por fim, para achar o $\gamma$, Wilmott subtrai o delta forward do delta backward e divide o resultado por $\delta S$, chegando na aproximação  $\frac{\partial V^2}{\partial S^2} (S, t) \approx \frac{V_{i+1}^{k} - 2 V_{i}^{k} + V_{i-1}^{k} }{\delta S^2}$. Como demonstrado em notas de aula, todos estes resultados podem ser checados aplicando a expansão de Taylor.

### 2.2. Condição Terminal e Payoffs

O valor da opção no vencimento é simplemente seu payoff. Assim, não é necessário resolver nada para $T$, apenas para $S$. Usando a notação de diferenças finitas, os payoffs desejados ficam sendo da forma:

- $V_{i}^{0}=ln(i\delta S)$
- $V_{i}^{0}=(ln(i\delta S))^2$
- $V_{i}^{0}=(i\delta S-K)^2$
- $V_{i}^{0}=\mathbf{1}_{i\delta S > K}$
- $V_{i}^{0}=max(i\delta S-K, 0)$

Como o Wilmott explica, o método de diferenças finitas começa de trás para frente, será destes valores que a iteração começará, como se estivéssemos calculando o preço de um derivativo por árvores binomiais, também de trás para frente.

### 2.3. Condições de Contorno

Quando estivermos percorrendo o Grid, necessitaremos definir o preço do derivativo em seus extremos, quando $S=0$ e $S=I \delta S$, onde $I$ é o ponto mais alto do Grid. Esta condição depende do instrumento que está sendo precificado. Para **todos** dos contratos, utilizaremos a seguinte condição para quando $S=0$:

$$V^{k}_{0} = (1 - r \delta t)V^{k-1}_{0}$$

Para a condição de contorno superior, utilizaremos para a **maioria dos contratos**:

$$V^{k}_{I} = 2V^{k}_{I-1} - V^{k}_{I-2}$$

Esta condição é adequada pois, a medida que $S \rightarrow \infty$, o $\Gamma$ da maioria dos contratos tende a zero. Porém, isso não é verdade para o contrato cujo payoff é descrito por $V_{i}^{0}=(i\delta S-K)^2$. Diferenciando novamente o [delta](https://nbviewer.jupyter.org/github/ucaiado/Replicating_Strategy/blob/master/UiraCaiadoEx01.ipynb) deste contrato, chegamos que o gamma será dado por $2 \cdot e^{(r + \sigma^2)(T-t)}$. Neste caso,
como o preço do ativo é dado por:

$$V_T = S_t^2  \cdot e^{(r + \sigma^2)(T-t)} - 2KS_t + K^2 e^{-r(T-t)} $$

A condição de contorno será:

$$V^{k}_{I} = I \delta S^2  \cdot e^{(r + \sigma^2)\delta t} - 2K\cdot I \delta S + K^2 e^{-r\delta t}$$



## 3. Implementando o Modelo

Nesta seção vou decsrever o método de diferenças finitas explícito e implementar os códigos necessários.

### 3.1 Método Explícito

Seguindo notas de aula, substituindo as diferenciações encontradas na equação parabólica mencionada anteriormente e reescrevendo os outros termos com a notação de diferenças finitas, temos que:

\begin{equation}
\begin{aligned}
\frac{\partial V}{\partial t} + a(S, t) \cdot \frac{\partial V^2}{\partial S^2} + b(S, t) \cdot \frac{\partial V}{\partial S} + c(S, t)\cdot V &= 0 \\
\frac{V_{i}^{k} - V_{i}^{k+1} }{\delta t} + a_{i}^{k} \cdot \frac{V_{i+1}^{k} - 2 V_{i}^{k} + V_{i-1}^{k} }{\delta S^2} + \\b_{i}^{k} \cdot \frac{V_{i+1}^{k} - V_{i-1}^{k} }{2 \delta S} + c_{i}^{k}\cdot V_{i}^{k} &= 0
\end{aligned}
\end{equation}

Rearranjando equação acima para isolar $V_{i}^{k+1}$ e renomeando alguns termos, ficamos com:

$$V_{i}^{k+1} = A_{i}^{k}V_{i-1}^{k} + \left( 1 + B_{i}^{k} \right)V_{i}^{k} + C_{i}^{k}V_{i+1}^{k}$$

Onde:

\begin{equation}
\begin{aligned}
A_{i}^{k} &= \nu_1 a_{i}^{k} - 0.5\nu_2 b_{i}^{k} \\
B_{i}^{k} &= -2\nu_1 a_{i}^{k} - \delta t c_{i}^{k} \\
C_{i}^{k} &= \nu_1 a_{i}^{k} + 0.5\nu_2 b_{i}^{k}
\end{aligned}
\end{equation}

Sendo que $\nu_1=\frac{\delta t}{\delta S ^2}$ e $\nu_2=\frac{\delta t}{\delta S}$. A equação acima está definida apenas entre $i=1, ..., I-1$. OS pontos restantes necessários para discretização vem das condições de contorno. Como nós conhecemos o valor terminal em $V_{i}^{0}$, podemos calcular o valor de $V_{i}^{1}$ e assim por diante. Como o valor do instrumento em $k+1$ só depende dos valores dele em $k$, chamamos este método de **método explícito**.

Por último, para que a solução deste método seja estável, é necessário que satisfaça $\delta t \, \leqslant \frac{\sigma S^2}{2a}$ e $\delta S \leqslant \frac{2a}{|b|}$.

### 3.2 Codificando

bla

### 3.3. Convergência do método explícito



## 4. Precificando Instrumentos

Nesta seção vou precificar alguns instrumentos e comparar com suas soluçòes analíticas.

### 4.1. Bla

### 4.3. Usando Volatilidade Incerta

## 5. Conclusão

bla bla bla



## 6. Últimas Considerações

Quando ...

_Style notebook and change matplotlib defaults_

In [3]:
#loading style sheet
from IPython.core.display import HTML
HTML(open('ipython_style.css').read())

In [4]:
#changing matplotlib defaults
%matplotlib inline
import seaborn as sns
sns.set_palette("deep", desat=.6)
sns.set_context(rc={"figure.figsize": (8, 4)})
sns.set_style("whitegrid")
sns.set_palette(sns.color_palette("PuBuGn_d", 10))